In [ ]:
import os
import json

import pandas as pd
from lightning_whisper_mlx import LightningWhisperMLX
from tqdm import tqdm

In [ ]:
def write_json(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f, ensure_ascii=False)

### 전처리 이상 데이터 선별 및 재 변환

In [ ]:
df = pd.read_csv("./1st/voc-result-whisper-large-v3.csv")
df

In [ ]:
def segment_text(df, max_len=100):
    for _, row in df.iterrows():
        print(row["file_name"])
        for i in range(0, len(row["text"]), max_len):
            print(row["text"][i : i + max_len])

        print("-----------------------------")

In [ ]:
reprocessing_list = [
    "1438481_5113_5113_20240502_135754.mp3",
    "1438504_5116_5116_20240502_140852.mp3",
    "1439285_5110_5110_20240503_093824.mp3",
    "1439507_5113_5113_20240503_111528.mp3",
    "1439523_5113_5113_20240503_111907.mp3",
    "1439596_5108_5108_20240503_115135.mp3",
    "1439827_5110_5110_20240503_135043.mp3",
    "1439853_5112_5112_20240503_140658.mp3",
    "1439865_5111_5111_20240503_141642.mp3",
]

In [ ]:
model = "large-v3"

path_voice = "./data/voices"
list_voice = tqdm(reprocessing_list)

whisper = LightningWhisperMLX(model=model, batch_size=12, quant=None)
result = []

for voice_file in list_voice:
    list_voice.set_description(voice_file)

    path_voice_file = os.path.join(path_voice, voice_file)
    translate = whisper.transcribe(audio_path=path_voice_file)

    resp = {"file_name": voice_file, "original": translate}

    write_json(resp, f"./data/output/{voice_file}.json")
    result.append(resp)

In [ ]:
import duckdb

df = duckdb.query("SELECT * FROM './data/output/*.json'").df()

df_file_name = df["file_name"]
df = pd.json_normalize(df["original"])
df = pd.concat([df_file_name, df], axis=1)
df

In [ ]:
del df["segments"]
df.to_csv("./data/output.csv", index=False)

In [ ]:
segment_text(df)

### 기존 전처리 데이터 업로드를 위한 작업

In [ ]:
df = pd.read_csv("./1st/voc-result-whisper-large-v3.csv")
df

In [ ]:
df["model_name"] = "large-v3"
df["extract_time"] = None

del df["language"]

In [ ]:
df

In [ ]:
df = pd.concat(
    [df["file_name"], df["model_name"], df["extract_time"], df["text"]], axis=1
)
df

In [ ]:
def add_quotes(val):
    if isinstance(val, str):
        return f'"{val}"'
    return val

In [ ]:
quoted_df = df.map(add_quotes)
quoted_df

In [ ]:
df.to_csv("voc-test-sample-2nd.csv", index=False, quotechar='"', quoting=1)

In [ ]:
quoted_df.to_csv("voc-test-sample-2nd.csv", index=False)